In [0]:
import time as time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
from sklearn.model_selection import GridSearchCV
from sklearn.kernel_ridge import KernelRidge
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [0]:
p = pd.read_csv('https://raw.githubusercontent.com/TZstatsADS/fall2019-project4-sec2-grp4/master/output/p.csv?token=ANFEHIYZQBNDJYCI5LZVVOC53TRC4')
q = pd.read_csv('https://raw.githubusercontent.com/TZstatsADS/fall2019-project4-sec2-grp4/master/output/q.csv?token=ANFEHIZ4XNEHIVHIBUWZZE253OEC2')
rating = pd.read_csv('https://raw.githubusercontent.com/TZstatsADS/fall2019-project4-sec2-grp4/master/data/ml-latest-small/ratings.csv?token=ANFEHI6OTIDZYNVKN4YXMES53OHN4')
b_bin = pd.read_csv('https://raw.githubusercontent.com/TZstatsADS/fall2019-project4-sec2-grp4/master/output/b_bin.csv?token=ANFEHI2IAUZMRP55ZXDKB5C5357EI')
mu = pd.read_csv('https://raw.githubusercontent.com/TZstatsADS/fall2019-project4-sec2-grp4/master/output/mu.csv?token=ANFEHI6TI46D3AIEQXCFLMK5357GU')
b_movie = pd.read_csv('https://raw.githubusercontent.com/TZstatsADS/fall2019-project4-sec2-grp4/master/output/b_movie.csv?token=ANFEHIZ5VLCO2XYBEOOY4LK5357KA')
b_user = pd.read_csv('https://raw.githubusercontent.com/TZstatsADS/fall2019-project4-sec2-grp4/master/output/b_user.csv?token=ANFEHI6JR3UB7ZHQQKKD6TK5357MW')
q.drop(['Unnamed: 0'], axis = 1, inplace = True)
q.index = range(1, 11)
rating.index = range(1,100837)
#import dataset

In [0]:
q_transpose = q.T
q_transpose_normalized = q_transpose / np.linalg.norm(q_transpose, axis = 1, keepdims = True)
#transpose and normalize q matrix
q_transpose_normalized.reset_index(inplace = True)
#preparing for later-on dataset merge
q_transpose_normalized.rename(columns = {"index": "movieId"}, inplace = True)
#rename from "index" to "movieId"

In [0]:
rating['movieId'] = rating['movieId'].astype(int)
#change movieId into integer

In [0]:
q_transpose_normalized['movieId'] = q_transpose_normalized['movieId'].astype(int)
#change movieId into integer

In [20]:
rating_merge = pd.merge(rating, q_transpose_normalized, on = 'movieId', how = 'left')
#merge 'rating' and 'q_transpose_normalized' datasets
dict_of_users = {k: v for k, v in rating_merge.groupby('userId')}
#separating all users into 610 cat
dict_of_users.get(1)[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]
#checking if user_1 is in the dictionary

,1,2,3,4,5,6,7,8,9,10
0,0.044342,0.199142,-0.463143,0.629428,0.147033,0.164932,-0.245353,-0.244816,0.210255,0.366787
1,0.162518,-0.131341,-0.430139,0.328224,-0.221395,-0.581172,0.217023,-0.274188,-0.074990,-0.385888
2,-0.250080,-0.178285,0.259185,0.564103,-0.179296,-0.064600,-0.336000,-0.318234,0.064943,0.515343
3,-0.252341,0.227907,-0.184571,-0.402325,-0.282473,0.286085,0.023196,-0.197273,-0.129677,0.685962
4,-0.157853,-0.042128,0.215942,0.344440,-0.448408,0.188921,-0.053558,-0.096609,0.169759,0.728187
...,...,...,...,...,...,...,...,...,...,...
227,-0.271034,-0.468704,0.348927,-0.235152,-0.429409,-0.383918,0.095001,0.274279,0.211075,0.263094
228,-0.320456,-0.495849,0.011117,-0.006909,0.437602,-0.383843,-0.380343,-0.390981,-0.119692,-0.024194
229,0.211059,0.206261,-0.178957,0.335596,-0.095601,-0.733847,-0.317841,-0.218986,0.238897,-0.120579
230,-0.410045,0.633286,-0.361732,-0.121613,-0.353321,-0.250072,0.002564,-0.258660,-0.120356,0.128077


In [0]:
def RMSE(n_users, method):
  vecList = list()
  for i in range(1, n_users + 1):
    X = dict_of_users.get(i)[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]
    y = dict_of_users.get(i)[['rating']]
    pred = KernelRidge(kernel = method).fit(X, y).predict(X)
    sq_diff = np.square(np.array(pred) - np.array(y))
    mean_sq_diff = sq_diff.mean()
    root_mean_sq_diff = np.sqrt(mean_sq_diff)
    vecList.append(root_mean_sq_diff)
  return vecList
#for iteration return RMSE

In [0]:
def List_Sum_Function(list, length): 
   if (length == 0): 
     return 0
   else: 
     return list[length - 1] + List_Sum_Function(list, length - 1)
#define function of sum of list values

In [23]:
## Train kernel ridge ##
train, test = train_test_split(rating_merge, test_size = 0.2)
#spliting dataset 20% test 80% train
dict_of_users_train = {k: v for k, v in train.groupby('userId')}
#separating all users into 610 cat
dict_of_users_test = {k: v for k, v in test.groupby('userId')}
#separating all users into 610 cat

dict_of_users_train.get(1)[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]
#checking if user_1 is in the train dictionary
dict_of_users_test.get(1)[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]
#checking if user_1 is in the train dictionary

,1,2,3,4,5,6,7,8,9,10
29,-0.206239,-0.523830,-0.546806,-0.054451,-0.249471,-0.001695,-0.295234,0.388699,-0.283910,-0.003408
18,-0.083017,-0.468885,-0.081085,0.555407,0.184070,0.172640,-0.044696,0.574965,0.242052,-0.057843
205,0.075087,-0.223729,0.031139,0.661550,0.030831,0.109050,0.008360,0.630566,-0.043070,0.305464
102,-0.127919,-0.171355,0.139073,0.282139,-0.049384,0.474148,-0.045862,-0.409093,0.251353,0.628838
74,0.162106,-0.041282,0.183639,0.005166,-0.615140,-0.088850,-0.339368,-0.075838,0.022312,0.656167
54,-0.056896,0.275269,0.188703,-0.325303,0.021671,0.074776,-0.201809,-0.275856,0.742656,-0.324247
101,0.185483,-0.585578,-0.219025,-0.237725,-0.113008,0.035016,-0.600291,-0.283091,-0.083758,0.238132
123,0.127159,-0.044645,-0.280569,0.403477,-0.400634,0.544555,-0.359241,-0.259460,0.243313,0.166439
173,0.322718,0.100756,0.632737,0.385000,-0.050091,0.013503,0.328713,0.327499,-0.107027,0.328124
9,0.467868,0.411276,0.110390,-0.434564,0.308308,-0.111044,0.214584,-0.308934,0.401976,0.021530


In [0]:
def RMSE_train(n_users, kernel, alpha, gamma, degree):
  vector_train = list()
  for m in range(1, n_users + 1):
    X_train = dict_of_users_train.get(m)[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]
    y_train = dict_of_users_train.get(m)[['rating']]
    pred = KernelRidge(alpha = alpha, gamma = gamma, degree = degree, kernel = kernel).fit(X_train, y_train).predict(X_train)
    sq_diff = np.square(np.array(pred) - np.array(y_train))
    mean_sq_diff = sq_diff.mean()
    root_mean_sq_diff = np.sqrt(mean_sq_diff)
    vector_train.append(root_mean_sq_diff)
  return vector_train
#define RMSE train function

In [0]:
def RMSE_test(n_users, kernel, alpha, gamma, degree):
  vector_test = list()
  for m in range(1, n_users + 1):
    X_train = dict_of_users_train.get(m)[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]
    y_train = dict_of_users_train.get(m)[['rating']]
    X_test = dict_of_users.get(m)[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]
    y_test = dict_of_users.get(m)[['rating']]
    pred = KernelRidge(alpha = alpha, gamma = gamma, degree = degree, kernel = kernel).fit(X_train, y_train).predict(X_test)
    sq_diff = np.square(np.array(pred) - np.array(y_test))
    mean_sq_diff = sq_diff.mean()
    root_mean_sq_diff = np.sqrt(mean_sq_diff)
    vector_test.append(root_mean_sq_diff)
  return vector_test
#define RMSE test function

In [0]:
## Kernel Ridge Tuning Parameters ##
KernelList = ["rbf", "polynomial", "laplacian", "linear", "sigmoid"]
GammaList = [0.00001, 0.0001, 0.01]
AlphaList = [0.00001, 0.0001, 0.01]

vecList2_train = list()
vecList2_test = list()
for i in KernelList:
  for j in GammaList:
    for k in AlphaList:
        RMSE_total_train = List_Sum_Function(RMSE_train(610, kernel = i, alpha = k, gamma = j, degree = 3), len(RMSE_train(610, kernel = i, alpha = k, gamma = j, degree = 3)))
        RMSE_total_test = List_Sum_Function(RMSE_test(610, kernel = i, alpha = k, gamma = j, degree = 3), len(RMSE_test(610, kernel = i, alpha = k, gamma = j, degree = 3)))
        vecList2_train.append(RMSE_total_train)
        vecList2_test.append(RMSE_total_test)      
#finding out the best parameters for both training and testing datasets
#laplacian distribution with gamma = 0.01 and alpha = 0.0001 is the optimal parameters for our kernel ridge model

In [0]:
## Train Kernel Ridge Final Model ##
def train_model(datasets, n_users, kernel, alpha, gamma, degree):
  vecList_train = list()
  for i in range(1, n_users + 1):
    X = datasets.get(i)[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]
    y = datasets.get(i)[['rating']]
    X_overall = q_transpose_normalized[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]
    pred = KernelRidge(alpha = alpha, gamma = gamma, degree = degree, kernel = kernel).fit(X, y).predict(X_overall)
    vecList_train.append(pred)
  return vecList_train

In [27]:
t0 = time.time()
train_model(dict_of_users_train, 610, kernel = 'laplacian', alpha = 0.0001, gamma = 0.01, degree = 3)
training_time = time.time() - t0
training_time
#training time = 45.278982400894165s

45.653467893600464

In [28]:
np.shape(train_model(dict_of_users_train, 610, kernel = 'laplacian', alpha = 0.0001, gamma = 0.01, degree = 3))
#check dimension of the final recommender system which is 610 * 9724

(610, 9724, 1)

In [0]:
def RMSE_aggregate(datasets):
  vecList_agg = list()
  for i in range(1, 610 + 1):
    X = datasets.get(i)[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]
    y = datasets.get(i)[['rating']]
    X_overall = dict_of_users.get(i)[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]
    y_overall = dict_of_users.get(i)[['rating']]
    pred = KernelRidge(alpha = 0.0001, gamma = 0.01, degree = 3, kernel = 'laplacian').fit(X, y).predict(X_overall)
    sq_diff = np.square(np.array(pred) - np.array(y_overall))
    sq_diff_sum = np.sum(sq_diff)
    vecList_agg.append(sq_diff_sum)
  return vecList_agg

In [30]:
np.sqrt(List_Sum_Function(RMSE_aggregate(dict_of_users), len(RMSE_aggregate(dict_of_users)))/610)

0.7806703963573395